<a href="https://colab.research.google.com/github/qkrqhdud/GMB_Gemma2-/blob/main/PPP/source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install torch
!pip install unsloth
!pip install bitsandbytes
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB

In [6]:
from transformers import pipeline
from unsloth import FastLanguageModel
import re

def load_model():
    finetune_model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "yunho123/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    )
    FastLanguageModel.for_inference(finetune_model)
    return finetune_model,tokenizer

def get_output(post_input,finetune_model,tokenizer):
  # 프롬프트
  post_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

  ### Input:
  {}

  ### Instruction:
  "Please rate the following blog post from 1 to 10 on each of the following category and provide a brief comment explaining your score for each criterion: "Topic Relevance", "Value Provision", "Storytelling", "Expertise","Conciseness","Paragraph Structure","Attention-Grabbing Opening", "Call-to-Action (CTA)","Balance of Professionalism and Personal Touch" and "Hashtag Usage"
   Please provide the output in the following format."

    "category": "Topic Relevance",
    "score": 5,
    "explanation": "comment",
    "improvement_suggestion": "comment"
    #next category
    "category": ,
    "score": ,
    "explanation": "comment",
    "improvement_suggestion": "comment"


  ### Response:
  {}"""
  inputs = tokenizer(
  [
      post_prompt.format(
          post_input,
          ""
      )
  ], return_tensors = "pt").to("cuda")

  outputs = tokenizer.batch_decode(finetune_model.generate(**inputs, max_new_tokens = 516, use_cache = True))

  response_start = outputs[0].find("### Response:")
  if response_start != -1:
    output = outputs[0][response_start+len("### Response:"):].strip()

  result = []
  matches = re.findall(r'"category":\s*"(.*?)",\s*\n\s*"score":\s*(.*?),\s*\n\s*"explanation":\s*"(.*?)",\s*\n\s*"improvement_suggestion":\s*"(.*?)"', output)

  for match in matches:
      category, score, explanation, improvement_suggestion = match
      result.append({"category": category, "score": int(score), "explanation": explanation, "improvement_suggestion": improvement_suggestion})


  # 결과 출력
  return result

In [7]:
model,tokenizer = load_model()
post_input = 'Agree with  Zach Boren , Johns post highlights the importance of finding opportunities for young people  #apprenticeshipworks'
#data 13 번
results = get_output(post_input,model,tokenizer)



==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
results

[{'category': 'Topic Relevance',
  'score': 8,
  'explanation': 'The blog post is highly relevant to the topic of apprenticeships and their importance for young people. It provides valuable insights and information on the subject, making it a valuable resource for anyone interested in the topic.',
  'improvement_suggestion': 'The post could be even more relevant by providing more specific examples of how apprenticeships can benefit young people and how they can be used to help them develop their skills and advance their careers.'},
 {'category': 'Value Provision',
  'score': 9,
  'explanation': 'The blog post provides a great deal of value to its readers by providing valuable information and insights on the topic of apprenticeships. It highlights the importance of apprenticeships for young people and provides practical advice on how to find and make the most of these opportunities.',
  'improvement_suggestion': 'The post could be even more valuable by providing more specific examples o